In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the labels for emotions
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Define the CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),  # Specify input shape
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(emotion_labels), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)


# Load and preprocess the dataset
def load_data():
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # For training and validation
    test_datagen = ImageDataGenerator(rescale=1./255)  # For testing

    train_data = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32,
        subset='training'
    )
    
    val_data = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32,
        subset='validation'
    )

    test_data = test_datagen.flow_from_directory(
        'dataset/test',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32
    )
    
    return train_data, val_data, test_data

# Train the model
train_data, val_data, test_data = load_data()
input_shape = (48, 48, 1)
model = create_cnn_model(input_shape)
model.fit(train_data, epochs=15, validation_data=val_data)

# Save the trained model
model.save('emotion_cnn_model.keras')


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/15


c:\Users\annsu\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


718/718 ━━━━━━━━━━━━━━━━━━━━ 23s 31ms/step - accuracy: 0.2480 - loss: 1.8242 - val_accuracy: 0.3761 - val_loss: 1.5797
Epoch 2/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 23s 32ms/step - accuracy: 0.3830 - loss: 1.5804 - val_accuracy: 0.4443 - val_loss: 1.4324
Epoch 3/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.4472 - loss: 1.4371 - val_accuracy: 0.4806 - val_loss: 1.3493
Epoch 4/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 22s 31ms/step - accuracy: 0.4806 - loss: 1.3573 - val_accuracy: 0.5095 - val_loss: 1.3004
Epoch 5/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 23s 32ms/step - accuracy: 0.5117 - loss: 1.2808 - val_accuracy: 0.5246 - val_loss: 1.2510
Epoch 6/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.5280 - loss: 1.2420 - val_accuracy: 0.5300 - val_loss: 1.2418
Epoch 7/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.5513 - loss: 1.1788 - val_accuracy: 0.5271 - val_loss: 1.2451
Epoch 8/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.5684 - loss: 1.1538 - val_accurac